In [1]:
# ---------------------------------- OPTIONS
! pip install pandas
! pip install openpyxl
! pip install xgboost
! pip install scikit-learn
! pip install iteration_utilities
! pip install ta
! pip install MetaTrader5
! pip install openpyxl
! pip install seaborn
! pip install plotly
! pip install ffn
! pip install quantstats
! pip install scikit-optimize
! pip install keras
! pip install pandas_ta
! pip install scikeras

In [3]:
# ---------------------------------- PACKAGES
import MetaTrader5 as mt5
# from keras.models import Sequential
# from keras.layers import LSTM
# from keras.layers import Dropout
# from keras.layers import Dense
from sklearn.feature_selection import RFECV
import ta
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from scipy.stats import norm
from sklearn import neighbors
import sys
from pprint import pprint
from sklearn.ensemble import GradientBoostingRegressor
from IPython import display
from skopt import BayesSearchCV
import pandas as pd
import plotly as pl
from datetime import datetime
import plotly.express as px
import plotly
import pandas as pd
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import RandomizedSearchCV
from statistics import stdev
from statistics import variance
import os
import glob
import numpy as np
import warnings
import plotly.graph_objs as go
import quantstats as qs
import plotly.subplots
import seaborn as sns
from bs4 import BeautifulSoup as BS
import urllib.request
import re
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from iteration_utilities import deepflatten
import pandas_ta as ta
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
# from scikeras.wrappers import KerasRegressor
# import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import GRU, Dense
# from keras.layers import LSTM, Dense
# import tensorflow
# import tensorflow.keras.wrappers
# from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.layers import Dropout
import pandas as pd
from skopt.space import Real, Integer, Categorical
from scipy.stats import loguniform
from scipy.stats import uniform
from scipy.stats import randint
from datetime import date

warnings.filterwarnings("ignore")

In [5]:
# ---------------------------------- Data Import
AUDUSD=pd.read_pickle('AUDUSD.pkl')
EURUSD=pd.read_pickle('EURUSD.pkl')
GBPUSD=pd.read_pickle('GBPUSD.pkl')
NZDUSD=pd.read_pickle('NZDUSD.pkl')
USDCAD=pd.read_pickle('USDCAD.pkl')
USDCHF=pd.read_pickle('USDCHF.pkl')

In [6]:
# --------------------------------- TA
def cci(pair,cci_sma,cci_num):
    cci_num=0.015
    pair['typ.price']=(pair['high']+pair['low']+pair['close'])/3
    pair['cci']=(pair['typ.price']-pair['typ.price'].rolling(cci_sma).mean())/(cci_num*stdev(pair['typ.price']))

def atr(pair,period):
    pair['atr']=ta.atr(pair['high'],pair['low'],pair['close'],period)
# def ichimoku(pair):
#     pair[['ISA1','ISB1','ITS1','IKS1','ICS1']]=ta.ichimoku(pair['high'],pair['low'],pair['close'],tenkan=True,kijun=True,senkou=True)
def willr(pair,period):
    pair['willr']=ta.willr(pair['high'],pair['low'],pair['close'],period)
def adx(pair,period):
    pair[['ADX_14','DMP_14','DMN_14']]=ta.adx(pair['high'],pair['low'],pair['close'],period)
def macd(pair,slow,fast,smooth): # 26,12,9
    pair['macd_slow']=pair['close'].ewm(span=fast).mean()-pair['close'].ewm(span=slow).mean()
    pair['macd_fast']=pair['close'].ewm(span=smooth).mean()
def Stochastic(pair,k_period,d_period):
    pair['sto_dif1']=pair['close']-pair['low'].rolling(k_period).min()
    pair['sto_dif2']=pair['high'].rolling(k_period).max()-pair['low'].rolling(k_period).min()
    pair['sto_k']=pair['sto_dif1']/pair['sto_dif2']*100
    pair['sto_main']=pair['sto_dif1'].rolling(d_period).sum()/pair['sto_dif2'].rolling(d_period).sum()*100
    pair['sto_signal']=pair['sto_main'].rolling(d_period).mean()
def MA(pair,period,type_ma):
    if type_ma=="sma":
        pair['sma'+str(period)]=pair['close'].rolling(period).mean()
    elif type_ma=="ema":
        pair['ema'+str(period)]=pair['close'].ewm(span=period).mean()
def RSI(pair, periods):
    pair['rsi_up']=pair['close']-pair['close'].shift(1) #shift(1)-previous, shift(-1)-next
    pair['rsi_down']=pair['close'].shift(1)-pair['close']
    pair.loc[pair['rsi_up'] <= 0, 'rsi_up'] = 0
    pair.loc[pair['rsi_down'] <= 0, 'rsi_down'] = 0
    pair['rsi_up_avg']=pair['rsi_up'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    pair['rsi_down_avg']=pair['rsi_down'].rolling(periods).mean()#.apply(lambda x: x[x!= 0].mean())
    for i in range(periods+1,len(pair)):
        pair['rsi_up_avg'][i]=(pair['rsi_up_avg'][i-1]*(periods-1)+pair['rsi_up'][i])/periods
        pair['rsi_down_avg'][i]=(pair['rsi_down_avg'][i-1]*(periods-1)+pair['rsi_down'][i])/periods
    pair['rsi_rs']=pair['rsi_up_avg']/pair["rsi_down_avg"]
    pair['rsi']=100-(100/(1+pair['rsi_rs']))
def BB(pair,avgday,stdmul):
    pair['bb_std']=pair['close'].rolling(avgday).std()
    pair['bb_mid']=pair['close'].rolling(avgday).mean()
    pair['bb_down']=pair['bb_mid']-pair['bb_std']*stdmul
    pair['bb_up']=pair['bb_mid']+pair['bb_std']*stdmul

for a in [EURUSD, AUDUSD, GBPUSD, NZDUSD,USDCAD,USDCHF]:
    MA(a,10,'ema')
    MA(a,20,'ema')
    MA(a,50,"ema")
    MA(a,200,"ema")
    MA(a,100,"ema")
    macd(a,26,12,9)
    RSI(a, 14)
    Stochastic(a,14,3)
    cci(a,20,0.015)
    BB(a,20,2)
    atr(a,14)
    willr(a,14)
    adx(a,14)
    a=a.drop(columns=['sto_dif1', 'sto_dif2','tick_volume', 'spread',
                      'real_volume','rsi_up', 'rsi_down', 'rsi_up_avg', 'rsi_down_avg',
                      'rsi_rs','sto_k','bb_std','typ.price'], axis=1, inplace=True)

In [7]:
# ---------------------------------- RETURN
for a in [EURUSD, AUDUSD, GBPUSD, NZDUSD,USDCAD,USDCHF]:
    a['close_pre']=a.close.shift(1)
    a['return']=(a['close']-a['close_pre'])/a['close_pre'] # simple return

In [8]:
EURUSD

,time,open,high,low,close,ema10,ema20,ema50,ema200,ema100,...,bb_mid,bb_down,bb_up,atr,willr,ADX_14,DMP_14,DMN_14,close_pre,return
0,2000-01-03,1.00730,1.02780,1.00540,1.02460,1.024600,1.024600,1.024600,1.024600,1.024600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,1.02430,1.03400,1.02130,1.02920,1.027130,1.027015,1.026946,1.026911,1.026923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.02460,0.004490
2,2000-01-05,1.02930,1.04020,1.02840,1.03260,1.029329,1.029066,1.028907,1.028827,1.028853,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.02920,0.003304
3,2000-01-06,1.03250,1.04150,1.02720,1.03300,1.030538,1.030202,1.029992,1.029886,1.029921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.03260,0.000387
4,2000-01-07,1.03290,1.03320,1.02600,1.02980,1.030326,1.030104,1.029951,1.029868,1.029896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.03300,-0.003098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6167,2023-09-25,1.06430,1.06557,1.05755,1.05921,1.066824,1.071811,1.081003,1.079072,1.083723,...,1.072658,1.055895,1.089420,0.006760,-91.398964,27.530634,10.252328,26.446560,1.06459,-0.005054
6168,2023-09-26,1.05931,1.06090,1.05622,1.05719,1.065073,1.070418,1.080069,1.078854,1.083198,...,1.071122,1.054616,1.087627,0.006612,-95.298110,28.874168,9.733969,26.546277,1.05921,-0.001907
6169,2023-09-27,1.05717,1.05741,1.04881,1.05031,1.062388,1.068503,1.078902,1.078570,1.082547,...,1.069021,1.053196,1.084846,0.006754,-94.650499,30.857650,8.848608,31.968746,1.05719,-0.006508
6170,2023-09-28,1.05009,1.05788,1.04912,1.05661,1.061338,1.067371,1.078028,1.078352,1.082033,...,1.067632,1.052630,1.082635,0.006897,-72.182596,32.554612,8.532592,29.068448,1.05031,0.005998
